# 迁移学习微调训练-可视化

#### 导入工具包

In [141]:
import time
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 获取计算硬件
# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:3


#### 图像预处理

In [142]:
from torchvision import transforms
from torch.utils.data import DataLoader


# 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

#### 载入数据集

In [143]:
from torchvision import datasets
batch_size=200
train_path = '/home/Datasets/FGVC/CUB/train'
test_path = '/home/Datasets/FGVC/CUB/test'
train_dataset = datasets.ImageFolder(train_path,train_transform)
test_dataset = datasets.ImageFolder(test_path,test_transform)
train_loader =DataLoader(test_dataset,shuffle=True,batch_size=batch_size,
                        num_workers=4)
test_loader =DataLoader(test_dataset,shuffle=False,batch_size=batch_size,
                        num_workers=4)

#### 选择迁移模型和训练方式

In [162]:
import torch
from torchvision import models
from  torch import nn,optim
import numpy

out_class = 200

model = models.resnet50(pretrained = True)
model.fc = nn.Linear(model.fc.in_features,out_class)
# 随机初始化模型全部权重，从头训练所有层
optimizer = optim.Adam(model.parameters())

# 模型加入 cuda
model = model.to(device)

criterion = nn.CrossEntropyLoss()


#### 函数在训练集上训练

In [163]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [164]:
from torch.optim import lr_scheduler
import numpy as np
Epoch =30

# 学习率降低策略：每隔5个epoch降低一半
lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.5)


# 运行一个 batch 的训练，返回当前 batch 的训练日志
def train_one_batch(images,labels):
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images)
    loss = criterion(outputs,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _,preds = torch.max(outputs,1)
    # 将数据转为numpy格式
    preds = preds.cpu().numpy()
    loss = loss.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    log_train ={}
    log_train['epoch']= epoch
    log_train['batch'] = batch_idx
    
    # 计算分类评估指标
    log_train['train_loss']=loss
    log_train['train_accuracy']=accuracy_score(labels,preds)
    
    # 返回训练日志
    return log_train 

#### 在整个数据集上评估

In [165]:
def evaluate_testset():
    '''
    在整个测试集上评估，返回分类评估指标日志
    '''

    loss_list = []
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_loader: # 生成一个 batch 的数据和标注
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) # 输入模型，执行前向预测

            # 获取整个测试集的标签类别和预测类别
            _, preds = torch.max(outputs, 1) # 获得当前 batch 所有图像的预测类别
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels) # 由 logit，计算当前 batch 中，每个样本的平均交叉熵损失函数值
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()
            # 数据添加至列表
            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)
        
    log_test = {}
    log_test['epoch'] = epoch
    
    # 计算分类评估指标
    log_test['test_loss'] = np.mean(loss)
    log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
    log_test['test_precision'] = precision_score(labels_list, preds_list, average='macro')
    log_test['test_recall'] = recall_score(labels_list, preds_list, average='macro')
    log_test['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')
    
    return log_test

#### 训练开始，记录日志

In [166]:
epoch = 0
batch_idx = 0
best_test_accuracy = 0

In [173]:
import pandas as pd
# 训练日志-训练集
df_train_log = pd.DataFrame() # pd格式
log_train = {}
log_train['epoch'] = 0
image,label = next(iter(train_loader))
log_train.update(train_one_batch(image,label))
df_train_log = df_train_log.append(log_train, ignore_index=True)

In [174]:
df_train_log

,epoch,batch,train_loss,train_accuracy
0,0,0,5.5454617,0.005


In [170]:
# 训练日志- 测试集
df_test_log = pd.DataFrame()
log_test ={}
log_test['epoch']=0
# 更新测试参数
log_test.update(evaluate_testset())
# 测试参数添加到字典
df_test_log = df_test_log.append(log_test,ignore_index=True)

In [171]:
df_test_log

,epoch,test_loss,test_accuracy,test_precision,test_recall,test_f1-score
0,0.0,5.656605,0.002854,0.000241,0.002998,0.000412


#### 登录wandb
1. 安装`wandb`: `pip install wandb`
2. 登录`wandb`: `wandb login`
3. 按照提示复制粘贴API key 到terminal中

#### 创建可视化wandb项目

In [181]:
import wandb
wandb.init(project='bird_200',name=time.strftime('%m-%d-%H:%M:%S'))

batch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
test_accuracy,▁▂▃▄▆▇█████████████████████████
test_f1-score,▁▂▃▄▆▇█████████████████████████
test_loss,█▆▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_precision,▁▃▃▅▆██████████████████████████
test_recall,▁▂▃▄▆▇█████████████████████████
train_accuracy,▁▁▂▃▄▅▆█████████████████████████████████
train_loss,█▇▅▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch,558
epoch,30


#### 运行训练

In [182]:
for epoch in range(1,Epoch+1):
    print(f'Epoch {epoch}/{Epoch}')
    # 训练阶段
    model.train()
    for images,labels in tqdm(train_loader):
        batch_idx += 1
        log_train = train_one_batch(images,labels)
        df_train_log =df_train_log.append(log_train,ignore_index=True)
        wandb.log(log_train)
    # 优化学习率
    lr_scheduler.step()
    
    # 测试阶段
    model.eval()
    log_test = evaluate_testset()
    # df 是 datafram 格式
    df_test_log = df_test_log.append(log_test,ignore_index=True)
    wandb.log(log_test)
    # 保存最新的最佳模型文件
    if log_test['test_accuracy'] > best_test_accuracy:
        # 删除旧的最佳模型文件
        old_best_checkpoint_path = '../Demo_zihao/checkpoints/best-{:.3f}.pth'.format(best_test_accuracy)
        if os.path.exists(old_best_checkpoint_path):
            os.remove(old_best_checkpoint_path)
        # 保存新的最佳模型
        new_best_check_path ='../Demo_zihao/checkpoints/best-{:.3f}.pth'.format(log_test['test_accuracy'])
        torch.save(model,new_best_check_path)
        print('保存最佳训练模型','/checkpoints/best-{:.3f}.pth'.format(best_test_accuracy))
        best_test_accuracy = log_test['test_accuracy']
        
df_train_log.to_csv('训练日志-训练集.csv',index=False)
df_train_log.to_csv('训练日志-测试集.csv',index=False)

Epoch 1/30


100%|███████████████████████████████████████████████████████████████████████████████████| 18/18 [00:09<00:00,  1.82it/s]


Epoch 2/30


100%|███████████████████████████████████████████████████████████████████████████████████| 18/18 [00:09<00:00,  1.82it/s]


Epoch 3/30


...

100%|███████████████████████████████████████████████████████████████████████████████████| 18/18 [00:09<00:00,  1.80it/s]


Epoch 29/30


100%|███████████████████████████████████████████████████████████████████████████████████| 18/18 [00:09<00:00,  1.80it/s]


Epoch 30/30


100%|███████████████████████████████████████████████████████████████████████████████████| 18/18 [00:09<00:00,  1.80it/s]


#### 在测试集上评价


In [184]:
# 载入最佳模型
#model = torch.load('../Demo_zihao/checkpoints/best-{:.3f}.pth'.format(best_test_accuracy))
model.eval()
print(evaluate_testset())

{'epoch': 30, 'test_loss': 0.005292853, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}
